## Neural Networks

Deep Learning is based on artificial neural networks which have been around in some form since the late 1950s. The networks are built from individual parts approximating neurons, typically called units or simply "neurons." Each unit has some number of weighted inputs. These weighted inputs are summed together (a linear combination) then passed through an activation function to get the unit's output.

<img src="assets/simple_neuron.png" width=400px>

Mathematically this looks like: 

$$
\begin{align}
y &= f(w_1 x_1 + w_2 x_2 + b) \\
y &= f\left(\sum_i w_i x_i \right)
\end{align}
$$

With vectors this is the dot/inner product of two vectors:

$$
h = \begin{bmatrix}
x_1 \, x_2 \cdots  x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_1 \\
           w_2 \\
           \vdots \\
           w_n
\end{bmatrix}
$$

### Stack them up!

We can assemble these unit neurons into layers and stacks, into a network of neurons. The output of one layer of neurons becomes the input for the next layer. With multiple input units and output units, we now need to express the weights as a matrix.

<img src='assets/multilayer_diagram_weights.png' width=450px>

We can express this mathematically with matrices again and use matrix multiplication to get linear combinations for each unit in one operation. For example, the hidden layer ($h_1$ and $h_2$ here) can be calculated 

$$
\vec{h} = [h_1 \, h_2] = 
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot 
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

The output for this small network is found by treating the hidden layer as inputs for the output unit. The network output is expressed simply

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$

## Tensors

It turns out neural network computations are just a bunch of linear algebra operations on *tensors*, a generalization of matrices. A vector is a 1-dimensional tensor, a matrix is a 2-dimensional tensor, an array with three indices is a 3-dimensional tensor (RGB color images for example). The fundamental data structure for neural networks are tensors and PyTorch (as well as pretty much every other deep learning framework) is built around tensors.

<img src="assets/tensor_examples.svg" width=600px>

With the basics covered, it's time to explore how we can use PyTorch to build a simple neural network.

In [1]:
import numpy as np
import torch

First, let's see how we work with PyTorch tensors. These are the fundamental data structures of neural networks and PyTorch, so it's imporatant to understand how these work.

In [2]:
x = torch.rand(5, 4)
x

tensor([[0.7724, 0.2895, 0.3457, 0.9053],
        [0.6386, 0.0626, 0.7364, 0.8477],
        [0.2926, 0.6341, 0.4619, 0.1237],
        [0.0563, 0.1429, 0.2170, 0.1049],
        [0.5624, 0.2284, 0.0077, 0.6710]])

In [3]:
y = torch.ones(x.size())
y

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [4]:
z = x + y
z

tensor([[1.7724, 1.2895, 1.3457, 1.9053],
        [1.6386, 1.0626, 1.7364, 1.8477],
        [1.2926, 1.6341, 1.4619, 1.1237],
        [1.0563, 1.1429, 1.2170, 1.1049],
        [1.5624, 1.2284, 1.0077, 1.6710]])

In general PyTorch tensors behave similar to Numpy arrays. They are zero indexed and support slicing.

In [5]:
z[-1]

tensor([1.5624, 1.2284, 1.0077, 1.6710])

In [6]:
z[:, [1]]

tensor([[1.2895],
        [1.0626],
        [1.6341],
        [1.1429],
        [1.2284]])

In [7]:
z[:-1, 1]

tensor([1.2895, 1.0626, 1.6341, 1.1429])

Tensors typically have two forms of methods, one method that returns another tensor and another method that performs the operation in place. That is, the values in memory for that tensor are changed without creating a new tensor. In-place functions are always followed by an underscore, for example `z.add()` and `z.add_()`.

In [8]:
# Return a new tensor z - .2
z.add(-0.2)

tensor([[1.5724, 1.0895, 1.1457, 1.7053],
        [1.4386, 0.8626, 1.5364, 1.6477],
        [1.0926, 1.4341, 1.2619, 0.9237],
        [0.8563, 0.9429, 1.0170, 0.9049],
        [1.3624, 1.0284, 0.8077, 1.4710]])

In [9]:
# z tensor is unchanged
z

tensor([[1.7724, 1.2895, 1.3457, 1.9053],
        [1.6386, 1.0626, 1.7364, 1.8477],
        [1.2926, 1.6341, 1.4619, 1.1237],
        [1.0563, 1.1429, 1.2170, 1.1049],
        [1.5624, 1.2284, 1.0077, 1.6710]])

In [10]:
# Add -.2 and update z tensor in-place
z.add_(-0.2)

tensor([[1.5724, 1.0895, 1.1457, 1.7053],
        [1.4386, 0.8626, 1.5364, 1.6477],
        [1.0926, 1.4341, 1.2619, 0.9237],
        [0.8563, 0.9429, 1.0170, 0.9049],
        [1.3624, 1.0284, 0.8077, 1.4710]])

In [11]:
# z has been updated
z

tensor([[1.5724, 1.0895, 1.1457, 1.7053],
        [1.4386, 0.8626, 1.5364, 1.6477],
        [1.0926, 1.4341, 1.2619, 0.9237],
        [0.8563, 0.9429, 1.0170, 0.9049],
        [1.3624, 1.0284, 0.8077, 1.4710]])

### Reshaping

Reshaping tensors is a really common operation. First to get the size and shape of a tensor use `.size()`. Then, to reshape a tensor, use `.resize_()`. Notice the underscore, reshaping is an in-place operation.

In [12]:
z.size()

torch.Size([5, 4])

In [13]:
z.resize_(4, 5)

tensor([[1.5724, 1.0895, 1.1457, 1.7053, 1.4386],
        [0.8626, 1.5364, 1.6477, 1.0926, 1.4341],
        [1.2619, 0.9237, 0.8563, 0.9429, 1.0170],
        [0.9049, 1.3624, 1.0284, 0.8077, 1.4710]])

In [14]:
z

tensor([[1.5724, 1.0895, 1.1457, 1.7053, 1.4386],
        [0.8626, 1.5364, 1.6477, 1.0926, 1.4341],
        [1.2619, 0.9237, 0.8563, 0.9429, 1.0170],
        [0.9049, 1.3624, 1.0284, 0.8077, 1.4710]])

## Numpy to Torch and back

Converting between Numpy arrays and Torch tensors is super simple and useful. To create a tensor from a Numpy array, use `torch.from_numpy()`. To convert a tensor to a Numpy array, use the `.numpy()` method.

In [17]:
a = np.random.rand(4,2)
a

array([[0.96574995, 0.27731111],
       [0.89435403, 0.85026381],
       [0.4347095 , 0.17496717],
       [0.23210828, 0.51178269]])

In [18]:
b = torch.from_numpy(a)
b

tensor([[0.9657, 0.2773],
        [0.8944, 0.8503],
        [0.4347, 0.1750],
        [0.2321, 0.5118]], dtype=torch.float64)

In [19]:
b.numpy()

array([[0.96574995, 0.27731111],
       [0.89435403, 0.85026381],
       [0.4347095 , 0.17496717],
       [0.23210828, 0.51178269]])

The memory is shared between the Numpy array and Torch tensor, so if you change the values in-place of one object, the other will change as well.

In [21]:
# Multiply PyTorch Tensor by 1/5, in place
b.mul_(1/5)

tensor([[0.1931, 0.0555],
        [0.1789, 0.1701],
        [0.0869, 0.0350],
        [0.0464, 0.1024]], dtype=torch.float64)

In [22]:
# Numpy array matches new values from Tensor
a

array([[0.19314999, 0.05546222],
       [0.17887081, 0.17005276],
       [0.0869419 , 0.03499343],
       [0.04642166, 0.10235654]])